In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from build_pandas import build_pandas
#import xgboost as xgb
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
import utils

C:\Users\47901\anaconda3\lib\site-packages\pandas\_testing.py:24: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  import pandas._libs.testing as _testing


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
location = "A"

In [3]:
x_train, x_test = utils.preprocess_category(location)
x_train.drop(["time", "date_calc"], axis=1, inplace=True)
x_test.drop(["date_calc"], axis=1, inplace=True)
x_train['date_forecast'] = (pd.to_datetime(x_train['date_forecast'], format='%Y') - pd.to_datetime('2000', format='%Y')).dt.total_seconds()
x_test['date_forecast_dt'] = x_test['date_forecast']
x_test['date_forecast'] = (pd.to_datetime(x_test['date_forecast'], format='%Y') - pd.to_datetime('2000', format='%Y')).dt.total_seconds()

print(x_train.columns)
print(x_train.shape)
x_train.head()

NameError: name 'utils' is not defined

In [5]:
print(x_test.columns)
x_test.head()

Index(['absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'ceiling_height_agl:m', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'cloud_base_agl:m', 'date_forecast', 'dew_or_rime:idx',
       'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
       'direct_rad_1h:J', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'hour_0', 'hour_1', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_2', 'hour_20', 'hour_21',
       'hour_22', 'hour_23', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7',
       'hour_8', 'hour_9', 'is_day:idx', 'is_in_shadow:idx', 'month_1',
       'month_10', 'month_11', 'month_12', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       '

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,date_forecast,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,date_forecast_dt
0,4.4,1.286,912.700012,0.0,0.0,1041.199951,736214400.0,0.0,271.700012,0.0,...,-11.185,0.0,273.799988,80.699997,30210.699219,4.0,2.2,3.4,-0.0,2023-05-01 00:00:00
1,4.3,1.287,912.700012,0.0,0.0,1054.800049,736215300.0,0.0,271.700012,0.0,...,-10.825,0.0,273.799988,77.000000,30003.599609,4.0,2.1,3.4,-0.0,2023-05-01 00:15:00
2,4.3,1.287,912.700012,0.0,0.0,1068.300049,736216200.0,0.0,271.600006,0.0,...,-10.360,0.0,273.799988,73.099998,29797.099609,3.9,2.1,3.3,-0.0,2023-05-01 00:30:00
3,4.3,1.287,912.700012,0.0,0.0,1081.900024,736217100.0,0.0,271.600006,0.0,...,-9.794,0.0,273.799988,69.000000,29618.599609,3.9,2.0,3.3,-0.0,2023-05-01 00:45:00
4,4.3,1.287,NaN,0.0,0.0,1095.400024,736218000.0,0.0,271.600006,0.0,...,-9.128,0.0,273.799988,64.500000,29507.500000,3.9,2.0,3.3,-0.0,2023-05-01 01:00:00


In [ ]:
h2o.init()

aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=list(x_train.drop('pv_measurement', axis=1).columns), y='pv_measurement', training_frame=h2o.H2OFrame(x_train))


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_352-debug"; OpenJDK Runtime Environment (build 1.8.0_352-debug-b08); OpenJDK 64-Bit Server VM (build 25.352-b08-debug, mixed mode)
  Starting server from /cluster/home/hansal/.local/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpzfgdy6kf
  JVM stdout: /tmp/tmpzfgdy6kf/h2o_hansal_started_from_python.out
  JVM stderr: /tmp/tmpzfgdy6kf/h2o_hansal_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,21 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_hansal_iko910
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.250 Gb
H2O_cluster_total_cores:,1
H2O_cluster_allowed_cores:,1
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
09:54:15.382: _train param, Dropping bad and constant columns: [snow_drift:idx, elevation:m]

██

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
aml.leader

In [ ]:
preds = aml.predict(h2o.H2OFrame(x_test))
